Starting this with a lookback of the teams that have won the whole thing 

In [1]:
import pandas as pd
import numpy as np

# Sample data creation for demonstration.
# Assuming the real CSV file is named 'basketball_games.csv' and contains the mentioned columns.
data = pd.read_csv('MNCAATourneyDetailedResults.csv')
# Creating a DataFrame from the sample data
df = pd.DataFrame(data)

# Filtering for seasons greater than 2010
filtered_df = df[df['Season'] > 2005]

# Finding the WTeamID with the maximum DayNum for each distinct Season
result_df = filtered_df.loc[filtered_df.groupby('Season')['DayNum'].idxmax(), ['Season', 'DayNum', 'WTeamID']]

result_df


,Season,DayNum,WTeamID
255,2006,154,1196
319,2007,154,1196
383,2008,154,1242
447,2009,154,1314
511,2010,154,1181
578,2011,154,1163
645,2012,154,1246
712,2013,154,1257
779,2014,154,1163
846,2015,154,1181


In [2]:
conf_df = pd.read_csv('MconferenceTourneyGames.csv')
# conf_df =  conf_df[conf_df['ConfAbbrev'] == "big_east"]
conf_df_filtered = conf_df[conf_df['Season'] > 2005]


unique_confs = conf_df_filtered[['Season', 'ConfAbbrev']].drop_duplicates()

# For each unique season and conference pair, find the champion, runner-up, semifinalists, and quarterfinalists.
def tournament_details(row, df):
    season = row['Season']
    conf = row['ConfAbbrev']
    conf_games = df[(df['Season'] == season) & (df['ConfAbbrev'] == conf)]
    max_day = conf_games['DayNum'].max()
    champ_game = conf_games[conf_games['DayNum'] == max_day]
    champion = champ_game['WTeamID'].values[0]
    runner_up = champ_game['LTeamID'].values[0]
    semifinal_day = max_day - 1
    quarterfinal_day = max_day - 2
    semifinalists = conf_games[conf_games['DayNum'] == semifinal_day]['LTeamID'].tolist()
    quarterfinalists = conf_games[conf_games['DayNum'] == quarterfinal_day]['LTeamID'].tolist()
    return pd.Series([champion, runner_up, semifinalists, quarterfinalists])

# Apply the function to each unique conference-season pair
unique_confs[['Champion', 'RunnerUp', 'Semifinalists', 'Quarterfinalists']] = unique_confs.apply(lambda row: tournament_details(row, conf_df_filtered), axis=1)

# Merge this tournament detail with the main result DataFrame on Season
final_df_enhanced = pd.merge(result_df, unique_confs, on=['Season'], how='left')

final_df_enhanced


,Season,DayNum,WTeamID,ConfAbbrev,Champion,RunnerUp,Semifinalists,Quarterfinalists
0,2006,154,1196,a_sun,1125,1252,"[1391, 1205]","[1144, 1190, 1273, 1194]"
1,2006,154,1196,a_ten,1462,1386,"[1396, 1200]","[1247, 1387, 1203, 1150]"
2,2006,154,1196,acc,1181,1130,"[1314, 1448]","[1268, 1274, 1438, 1301]"
3,2006,154,1196,aec,1107,1436,[],[]
4,2006,154,1196,big_east,1393,1338,"[1437, 1207]","[1266, 1452, 1163, 1353]"
...,...,...,...,...,...,...,...,...
523,2023,154,1163,summit,1331,1295,"[1355, 1472]","[1377, 1442]"
524,2023,154,1163,sun_belt,1418,1375,"[1241, 1402]","[1407, 1379, 1267, 1204]"
525,2023,154,1163,swac,1411,1212,"[1238, 1105]","[1380, 1341]"
526,2023,154,1163,wac,1213,1381,"[1358, 1430]","[1370, 1465, 1469, 1470]"


In [3]:
tournament_results_df = unique_confs[['Season', 'ConfAbbrev', 'Champion', 'RunnerUp', 'Semifinalists', 'Quarterfinalists']].copy()

def determine_tournament_position_v2(row):
    season = row['Season']
    team_id = row['WTeamID']
    tournament_info = tournament_results_df[tournament_results_df['Season'] == season]

    # Initialize variables
    conf = None
    position = None

    # Check for champion, runner-up, semifinalist, quarterfinalist by iterating through the rows
    for _, t_row in tournament_info.iterrows():
        # Direct match for Champion and RunnerUp
        if team_id == t_row['Champion']:
            conf = t_row['ConfAbbrev']
            position = 'Champion'
            break
        elif team_id == t_row['RunnerUp']:
            conf = t_row['ConfAbbrev']
            position = 'RunnerUp'
            break
        # Check within lists for Semifinalists and Quarterfinalists
        elif any(team_id == semi_id for semi_id in t_row['Semifinalists']):
            conf = t_row['ConfAbbrev']
            position = 'Semifinalist'
            break
        elif any(team_id == quarter_id for quarter_id in t_row['Quarterfinalists']):
            conf = t_row['ConfAbbrev']
            position = 'Quarterfinalist'
            break

    return pd.Series([conf, position])

# Re-apply the corrected function to the results dataframe
result_df[['ConfAbbrev', 'TournamentResult']] = result_df.apply(determine_tournament_position_v2, axis=1)

result_df

,Season,DayNum,WTeamID,ConfAbbrev,TournamentResult
255,2006,154,1196,sec,Champion
319,2007,154,1196,sec,Champion
383,2008,154,1242,big_twelve,Champion
447,2009,154,1314,acc,Semifinalist
511,2010,154,1181,acc,Champion
578,2011,154,1163,big_east,Champion
645,2012,154,1246,sec,RunnerUp
712,2013,154,1257,big_east,Champion
779,2014,154,1163,aac,RunnerUp
846,2015,154,1181,acc,Semifinalist


In [4]:
df = final_df_enhanced

# Remove the 'WTeamID' and 'DayNum' columns
df.drop(columns=['WTeamID', 'DayNum'], inplace=True)

# Initialize a list to hold the new row-wise data
rows = []

# Iterate over each row in the dataframe to transform and accumulate the data
for _, row in df.iterrows():
    season, conf = row['Season'], row['ConfAbbrev']
    # Champion
    rows.append([season, conf, 'Champion', row['Champion']])
    # RunnerUp
    rows.append([season, conf, 'RunnerUp', row['RunnerUp']])
    # Semifinalists
    for team in row['Semifinalists']:
        rows.append([season, conf, 'Semifinalist', team])
    # Quarterfinalists
    for team in row['Quarterfinalists']:
        rows.append([season, conf, 'Quarterfinalist', team])

# Create a new DataFrame from the rows list
df_transformed = pd.DataFrame(rows, columns=['Season', 'ConfAbbrev', 'ConfTournamentPlace', 'TeamID'])

df_transformed.head(200)

,Season,ConfAbbrev,ConfTournamentPlace,TeamID
0,2006,a_sun,Champion,1125
1,2006,a_sun,RunnerUp,1252
2,2006,a_sun,Semifinalist,1391
3,2006,a_sun,Semifinalist,1205
4,2006,a_sun,Quarterfinalist,1144
...,...,...,...,...
195,2007,a_sun,Quarterfinalist,1391
196,2007,a_sun,Quarterfinalist,1273
197,2007,a_ten,Champion,1203
198,2007,a_ten,RunnerUp,1348


In [6]:
games_df = data

team_season_games = games_df.groupby(['Season', 'WTeamID']).size().reset_index(name='Wins').rename(columns={'WTeamID': 'TeamID'})
team_season_losses = games_df.groupby(['Season', 'LTeamID']).size().reset_index(name='Losses').rename(columns={'LTeamID': 'TeamID'})

# Merge the win and loss counts on Season and TeamID, filling missing values with 0
team_games_combined = pd.merge(team_season_games, team_season_losses, on=['Season', 'TeamID'], how='outer').fillna(0)

# Calculate total games played
team_games_combined['GamesPlayed'] = team_games_combined['Wins'] + team_games_combined['Losses']

# Drop the Wins and Losses columns as they are not required for the final output
team_games_combined.drop(columns=['Wins', 'Losses'], inplace=True)

team_games_combined.head()

,Season,TeamID,GamesPlayed
0,2003,1112,4.0
1,2003,1113,2.0
2,2003,1120,3.0
3,2003,1139,3.0
4,2003,1141,2.0
